## **STEP1. 랭체인으로 GPT 부르기**

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

📌 핵심 개념:
- PromptTemplate: 변수를 포함한 프롬프트 정의
- system 메시지: AI의 역할/성격 지정
- human 메시지: 사용자의 실제 입력
- 템플릿을 쓰면 프롬프트 재사용이 쉬워짐

In [2]:
load_dotenv("key.env")

llm=ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
prompt=ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 선생님입니다. 초등학생도 이해할 수 있게 설명해주세요."),
    ("user", "{question}"),
])

#prompt와 llm을 파이프로 연결
chain = prompt | llm

#llm 단독호출
response = chain.invoke("양자역학이 뭐야?")
print(response.content)

좋은 질문이야! 양자역학은 아주아주 작은 세상, 예를 들어 원자나 전자처럼 눈에 보이지 않는 작은 입자들이 어떻게 움직이고 행동하는지를 연구하는 과학이야.

우리가 일상에서 보는 큰 물건들은 보통 뉴턴의 법칙으로 잘 설명할 수 있는데, 아주 작은 세계에서는 그 법칙들이 잘 맞지 않아. 그래서 과학자들이 새롭게 만든 법이 바로 양자역학이야.

양자역학에서는 작은 입자들이 마치 파도처럼 움직이기도 하고, 동시에 여러 곳에 있을 수도 있어. 그리고 우리가 그 입자를 관찰하면 그때서야 위치가 확실해진단다.

쉽게 말하면, 양자역학은 아주 작은 세계의 신기한 규칙들을 알려주는 과학이라고 생각하면 돼!


In [3]:
!pip install -U langchain-google-genai

In [4]:
from getpass import getpass

In [5]:
# Gemini API Key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Gemini API Key: ")

print("✅ OpenAI & Gemini API Key 설정 완료!")

✅ OpenAI & Gemini API Key 설정 완료!


| 패키지                   | 역할                                       |
| --------------------- | ---------------------------------------- |
| langchain-core      | **핵심 추상 개념 (Prompt, Runnable, Message)** |
| langchain           | 체인, 메모리, 에이전트 구현                         |
| langchain-openai    | OpenAI 연동                                |
| langchain-community | 서드파티 도구                                  |

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)

response = llm.invoke("안녕?")
print(response.content)

안녕!


## **Step 2: 메모리 관리 (대화 기록)**

[핵심 개념]
- ChatMessageHistory: 대화 기록을 저장하는 클래스
- RunnableWithMessageHistory: 메모리를 포함한 체인
- session_id: 대화 세션을 구분하는 ID
- store: 여러 세션의 대화 기록을 저장하는 딕셔너리

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [8]:
#1. llm
llm = ChatOpenAI(model_name="gpt-4.1-mini")

#2. prompt (history를 끼워 넣는 것이 핵심)
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

#3. 세션별 대화 저장소
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#4. chain
chain = prompt | llm

#5. memory를 체인에 연결
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_variable_name="history"
)

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 이전 대화를 기억하는 AI입니다."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chain = prompt | llm

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 세션1: 첫 번째 대화
print("=== 세션1 시작 ===")
config1 = {"configurable": {"session_id": "session1"}}

r1 = chain_with_memory.invoke({"input": "내 이름은 철수야"}, config=config1)
print(f"AI: {r1.content}")

r2 = chain_with_memory.invoke({"input": "내 이름이 뭐였지?"}, config=config1)
print(f"AI: {r2.content}")

# 세션2: 새로운 대화 (이전 내용 기억 못함)
print("\n=== 세션2 시작 (다른 세션) ===")
config2 = {"configurable": {"session_id": "session2"}}

r3 = chain_with_memory.invoke({"input": "내 이름이 뭐였지?"}, config=config2)
print(f"AI: {r3.content}")
print("\n결과: 세션2는 세션1의 대화를 기억하지 못함")

# 다시 세션1로 돌아가면 기억함
print("\n=== 세션1로 복귀 ===")
r4 = chain_with_memory.invoke({"input": "내 이름 다시 말해줘"}, config=config1)
print(f"AI: {r4.content}")
print("\n결과: 세션1은 여전히 '철수'를 기억함")

=== 세션1 시작 ===
AI: 안녕하세요, 철수님! 어떻게 도와드릴까요?
AI: 당신의 이름은 철수입니다. 어떻게 도와드릴까요?

=== 세션2 시작 (다른 세션) ===
AI: 죄송하지만 이전에 당신의 이름을 알려주신 적이 없어서 기억하지 못해요. 이름을 알려주시면 기억할게요!

결과: 세션2는 세션1의 대화를 기억하지 못함

=== 세션1로 복귀 ===
AI: 당신의 이름은 철수입니다.

결과: 세션1은 여전히 '철수'를 기억함


# Step 3: Tool 추가

In [10]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 파일 삭제
if os.path.exists("todos.txt"):
    os.remove("todos.txt")

# GPT에게 파일 저장 요청
response = llm.invoke("todos.txt에 '파이썬 공부'를 추가해줘")
print(f"GPT 답변: {response.content}")

# 실제 확인
if os.path.exists("todos.txt"):
    print("파일 있음")
else:
    print("파일 없음 - GPT는 말만 했음")

GPT 답변: `todos.txt` 파일에 '파이썬 공부'를 추가하려면, 다음과 같이 하면 됩니다:

```python
with open('todos.txt', 'a', encoding='utf-8') as file:
    file.write('파이썬 공부\n')
```

이 코드는 `todos.txt` 파일을 열어서(없으면 새로 만들고), 파일 끝에 '파이썬 공부'라는 문장을 한 줄 추가합니다. `encoding='utf-8'`은 한글이 깨지지 않도록 설정한 것입니다.
파일 없음 - GPT는 말만 했음


In [11]:
from langchain_core.tools import tool

In [12]:
# step4_define_tools.py

from langchain_core.tools import tool

@tool
def add_todo(task: str) -> str:
    """할 일을 todos.txt에 추가합니다"""
    with open("todos.txt", "a", encoding="utf-8") as f:
        f.write(f"[ ] {task}\n")
    return f"{task} 추가됨"

@tool
def show_todos() -> str:
    """할 일 목록을 보여줍니다"""
    try:
        with open("todos.txt", "r", encoding="utf-8") as f:
            return f"할 일:\n{f.read()}"
    except:
        return "할 일 없음"

# Tool 확인
print(f"Tool 이름: {add_todo.name}")
print(f"Tool 설명: {add_todo.description}")

# 테스트
print(add_todo.invoke({"task": "파이썬 공부"}))
print(show_todos.invoke({}))


Tool 이름: add_todo
Tool 설명: 할 일을 todos.txt에 추가합니다
파이썬 공부 추가됨
할 일:
[ ] 파이썬 공부



In [13]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
당신은 할 일 관리 AI입니다.

사용 가능한 도구:
{tools}

형식:
Question: {input}
Thought: 생각
Action: [{tool_names}]
Action Input: 입력
Observation: 결과
...
Final Answer: 답변

{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)
print(f"입력 변수: {prompt.input_variables}")

입력 변수: ['agent_scratchpad', 'input', 'tool_names', 'tools']


In [14]:
pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -U langchain langchain-community langchain-openai langchain-core langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv("key.env")

# -------------------------
# 1. LLM 정의
# -------------------------
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# -------------------------
# 2. Tools 정의
# -------------------------
@tool
def add_todo(task: str) -> str:
    """할 일 목록 파일(todos.txt)에 새로운 할 일을 추가합니다."""
    with open("todos.txt", "a", encoding="utf-8") as f:
        f.write(f"[ ] {task}\n")
    return f"'{task}' 내용이 추가되었습니다."

@tool
def show_todos() -> str:
    """todos.txt 파일에 저장된 모든 할 일 목록을 읽어서 보여줍니다."""
    try:
        if not os.path.exists("todos.txt"):
            return "할 일 목록이 비어있습니다."
        with open("todos.txt", "r", encoding="utf-8") as f:
            content = f.read()
            return f"현재 할 일 목록:\n{content}" if content else "할 일이 없습니다."
    except Exception as e:
        return f"파일 읽기 에러: {str(e)}"

tools = [add_todo, show_todos]

# -------------------------
# 3. Agent 생성
# -------------------------
# [수정] state_modifier -> prompt 로 변경
# (설치된 라이브러리 버전에 따라 파라미터 이름이 다릅니다)
agent_app = create_react_agent(
    model=llm,
    tools=tools,
    prompt="당신은 할 일 관리 도우미입니다. 사용자의 요청을 정확히 수행하세요." 
)

# -------------------------
# 4. 실행
# -------------------------
print("--- 에이전트 실행 중... ---")

# invoke 호출 (messages 리스트로 전달)
response = agent_app.invoke({
    "messages": [("human", "파이썬 공부를 할 일에 추가해줘")]
})

# -------------------------
# 5. 결과 출력
# -------------------------
print("\n[최종 답변]")
print(response["messages"][-1].content)

/tmp/ipykernel_32191/1728993194.py:44: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_app = create_react_agent(


--- 에이전트 실행 중... ---

[최종 답변]
'파이썬 공부'가 할 일 목록에 추가되었습니다. 다른 도움이 필요하신가요?


In [17]:
!pip install -U ddgs

  Using cached lxml-6.0.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl.metadata (3.6 kB)
Using cached lxml-6.0.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl (5.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [ddgs]5/9 [hpack]


In [21]:
from ddgs import DDGS

query = "2025년 현대자동차 미국 시장 전망"

print(f"검색어: {query}\n")

try:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        for i, r in enumerate(results, 1):
            print(f"[결과 {i}]")
            print(f"제목: {r['title']}")
            print(f"내용: {r['body'][:200]}...")
            print("-" * 60)
except Exception as e:
    print(f"검색 실패: {e}")

검색어: 2025년 현대자동차 미국 시장 전망

[결과 1]
제목: KRVIP 로그인 에서의 위험 요소와 예방 전략 - 2025년 최신 가이드
내용: 미국 전기차 테슬라가 조만간 국내에 6인승 전기 스포츠유틸리티(SUV) 차종인 '모델YL'을 선보이면서 국내 패밀리카 시장에 도전장을 내밀 것으로 보인다....
------------------------------------------------------------
[결과 2]
제목: 글로벌오토뉴스 Global Auto News (@global_auto_news) - Instagram
내용: 북미 자동차 전문 기자 50명이 1년간 테스트한 결과입니다. 왜 하이브리드 SUV가 최종승자가 되었을까요? 미국 시장의 변화 2025년 2분기 하이브리드 판매 36% 급증 ......
------------------------------------------------------------
[결과 3]
제목: [PDF] Future Trend and New Engine Technologies to Meet Emission ...
내용: ❖ 중국과 미국은 세계 전기차 시장을 선도하고 있음. ❖ 중국은 신 에너지 ... Source : 현대자동차 수소 연료전지, NH 투자증권 리서치 본부. (DOE, 2018). FCEV ......
------------------------------------------------------------


In [22]:
# example1_web_search.py
"""
[기능] 웹 검색 + GPT 요약
- ddgs로 실시간 검색
- 검색 결과를 한글로 요약
"""

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from ddgs import DDGS

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

@tool
def web_search(query: str) -> str:
    """웹에서 검색합니다"""
    try:
        results = []
        with DDGS() as ddgs:
            for r in ddgs.text(query, max_results=3):
                results.append(f"{r['title']}: {r['body']}")
        return "\n\n".join(results) if results else "결과 없음"
    except Exception as e:
        return f"검색 실패: {str(e)}"

tools = [web_search]
llm_with_tools = llm.bind_tools(tools)

print("웹 검색 봇 (quit로 종료)")

while True:
    user_input = input("\n> ").strip()

    if user_input.lower() in ['quit', 'exit', 'q', '종료', '끝']:
        break

    if not user_input:
        continue

    # LLM 호출
    messages = [{"role": "user", "content": user_input}]
    response = llm_with_tools.invoke(messages)

    # 도구 실행
    if response.tool_calls:
        search_result = web_search.invoke(response.tool_calls[0]["args"])

        # 요약
        summary = llm.invoke(f"다음 검색 결과를 한글로 3-4문장 요약:\n{search_result[:2000]}")
        print(f"\n{summary.content}")
    else:
        print(f"\n{response.content}")

웹 검색 봇 (quit로 종료)

발렌타인데이 선물로는 어떤 스타일이나 취향을 고려하고 계신가요? 예를 들어, 로맨틱한 선물, 실용적인 선물, 혹은 특별한 경험을 선물하고 싶은지 알려주시면 더 맞춤형 추천을 드릴 수 있어요. 아니면 선물 받을 분의 나이, 성별, 취미 등도 알려주시면 도움이 됩니다!

초콜릿을 제외하고 색다른 간식이나 디저트를 찾으시는 건가요? 아니면 색다른 선물이나 아이디어를 원하시는 건가요? 어떤 종류의 색다른 것을 원하시는지 조금 더 구체적으로 알려주시면 도움이 될 것 같아요!

간식을 제외한 어떤 내용을 원하시는지 구체적으로 알려주시면 도와드리겠습니다. 예를 들어, 식사 메뉴, 운동 계획, 공부 방법 등 어떤 주제에 대해 간식을 제외하고 알고 싶으신가요?

여자친구 생일 선물로 배스 낚시를 시도하는 영상이 TikTok에 올라왔으며, 실제로 배스를 잡는 장면이 포함되어 있습니다. 20대가 좋아하는 선물 아이디어도 TikTok에서 소개되고 있는데, 기념일이나 생일에 적합한 추천 리스트가 제공됩니다. 한편, 인스타그램에서는 Bosch 유리거울 청소기 등 집안일을 편리하게 해주는 신기한 가전제품들이 소개되고 있습니다.


In [27]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

# 1. 환경 설정
load_dotenv("key.env")

# 2. 도구(Tool) 정의: 성균관대 공지사항 크롤러
@tool
def fetch_skku_notices(keyword: str) -> str:
    """
    성균관대학교 학부 공지사항 사이트에서 키워드로 공지사항을 검색하여 리스트를 반환합니다.
    '장학금', '등록금', '수강신청' 등 학교 소식이 궁금할 때 사용하세요.
    """
    # 성균관대 공지사항 검색 URL (키워드 파라미터 포함)
    url = f"https://www.skku.edu/skku/campus/skk_comm/notice01.do?mode=list&srSearchVal={keyword}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 공지사항 목록 추출 (사이트 구조에 따른 선택자 설정)
        notice_list = soup.select(".board-list-wrap table tbody tr")
        
        if not notice_list:
            return f"'{keyword}'에 대한 검색 결과가 없습니다."
        
        results = []
        for item in notice_list[:10]:  # 최신 10개만 추출
            title_tag = item.select_one(".td-subject a")
            date_tag = item.select_one(".td-date")
            
            if title_tag:
                title = title_tag.get_text(strip=True)
                link = "https://www.skku.edu/skku/campus/skk_comm/notice01.do" + title_tag['href']
                date = date_tag.get_text(strip=True) if date_tag else "날짜 불명"
                results.append(f"📌 제목: {title}\n📅 날짜: {date}\n🔗 링크: {link}")
        
        return "\n\n".join(results)
    
    except Exception as e:
        return f"공지사항을 가져오는 중 오류가 발생했습니다: {str(e)}"

# 3. Agent 설정
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
tools = [fetch_skku_notices]

# 4. Streamlit UI
st.set_page_config(page_title="SKKU 공지 비서", page_icon="🏫")
st.title("🏫 성균관대 실시간 공지사항 비서")

# 사이드바: 내 정보(필터링용)
with st.sidebar:
    st.header("👤 나의 프로필")
    user_info = st.text_area("나의 조건 (학과, 학년, 관심분야)", 
                            value="컴퓨터공학과 3학년, 성적 3.8, 장학금에 관심 많음.")

# 세션 상태 초기화
if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "어떤 공지사항을 찾아드릴까요? (예: 장학금, 졸업, 인턴)"}]

for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# 사용자 입력 처리
if prompt := st.chat_input("키워드를 입력하세요..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # 에이전트 실행
    # 사용자의 프로필 정보를 프롬프트에 주입하여 맞춤형 요약을 유도함
    prompt_template = f"""
    당신은 성균관대학교 학생을 돕는 'SKKU 공지 비서'입니다.
    
    [학생 정보]
    {user_info}
    
    [임무]
    1. 사용자가 묻는 키워드로 'fetch_skku_notices' 도구를 사용해 공지를 검색하세요.
    2. 검색된 공지 내용이 [학생 정보]에 해당되는지 분석하세요.
    3. 지원 가능한 장학금이나 유용한 정보가 있다면 강조해서 설명해주세요.
    4. 친절하게 학교 선배처럼 답변하세요.
    """

    agent = create_react_agent(model=llm, tools=tools, prompt=prompt_template)

    with st.chat_message("assistant"):
        with st.spinner("학교 홈페이지에서 공지를 찾는 중..."):
            response = agent.invoke({"messages": [HumanMessage(content=prompt)]})
            final_answer = response["messages"][-1].content
            st.markdown(final_answer)

    st.session_state.messages.append({"role": "assistant", "content": final_answer})

2026-02-11 14:08:30.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 14:08:30.684 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 14:08:30.684 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 14:08:30.685 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 14:08:30.686 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 14:08:30.686 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 14:08:30.687 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 14:08:30.687 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar